In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd

# Binary

In [2]:
data_table = pd.read_csv('iris.csv', names =['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'flower'])

In [3]:
data_table.head()

,sepal_length,sepal_width,petal_length,petal_width,flower
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
xMatrix = np.array(data_table[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])
yVector = np.array(data_table['flower'])

In [5]:
def normalization(matrix, vector):
    #Mean calculation
    mVector = np.mean(vector, axis=0)
    mMatrix = np.mean(matrix, axis=0)
    
    #Standard deviation
    stdVector = np.std(vector, axis = 0)
    stdMatrix = np.std(matrix, axis = 0)
     
    #Normalization of data
    vector = (vector - mVector)/stdVector
    matrix = (matrix - mMatrix)/stdMatrix
    
    return matrix, vector

In [13]:
def gradientDescent(xMatrix, yVector, epochs, alpha):
    xMatrix = np.insert(xMatrix, 0, 1, axis =1)
    wCoefMatrix = np.ones((xMatrix.shape[1],), dtype=float)
    yPredicted = xMatrix.dot(wCoefMatrix)
    mse = []
    for epoch in range(epochs):
        somatorio = 0
        for i in range(xMatrix.shape[0]):
            somatorio += (yVector[i] - sigmoide(yPredicted[i])) * xMatrix[i]

        mse.append((-1/xMatrix.shape[0]) * somatorio)
        wCoefMatrix = wCoefMatrix + (alpha/wCoefMatrix.shape[0])*somatorio
        yPredicted = xMatrix.dot(wCoefMatrix)
    
    return wCoefMatrix

def stochasticGradientDescent(xMatrix, yVector, epochs, alpha):
    xMatrix = np.insert(xMatrix, 0, 1, axis =1)
    wCoefMatrix = np.ones((xMatrix.shape[1],), dtype=float)
    yPredicted = xMatrix.dot(wCoefMatrix)
    data = np.append(xMatrix, np.split(yVector, xMatrix.shape[0], axis=0), axis=1)
    mse = []

    for epoch in range(epochs):
        erro = 0
        somatorio = 0
        for i in range(xMatrix.shape[0]):
            wCoefMatrix  = wCoefMatrix + alpha * (yVector[i] - sigmoide(yPredicted[i])) * xMatrix[i]
            somatorio += (yVector[i] - sigmoide(yPredicted[i])) * xMatrix[i]
        
        mse.append((-1/xMatrix.shape[0]) * somatorio)  
        
        data = np.random.permutation(data)
        xMatrix = data[: ,0:xMatrix.shape[1]]
        yVector = data[:,xMatrix.shape[1]]
        
        yPredicted = xMatrix.dot(wCoefMatrix)  
        
    return wCoefMatrix

In [7]:
def crossValidationSplit(xMatrix, yVector, n_folds): 
    xMatrixSplit = []
    yVectorSplit = []
    xMatrixCopy = xMatrix

    yVectorCopy = yVector
    foldSize = int(len(xMatrix) / n_folds) 
    
    for i in range(n_folds): 
        foldXM = [] 
        foldYV = []
        while len(foldXM) < foldSize: 
            index = rd.randrange(len(xMatrixCopy)) 
            foldXM.append(xMatrixCopy[index]) 
            xMatrixCopy = np.delete(xMatrixCopy, index, axis=0)
            foldYV.append(yVectorCopy[index]) 
            yVectorCopy = np.delete(yVectorCopy, index)
        xMatrixSplit.append(foldXM)
        yVectorSplit.append(foldYV)
    return xMatrixSplit, yVectorSplit

def kFoldCrossValidation(xMatrix, yVector, kParts, algorithm, *args):
    xMFolds, yVFolds = crossValidationSplit(xMatrix, yVector, kParts) 
    scores = list() 
    count = 0;
    for fold in xMFolds: 
        xTrain = xMFolds
        yTrain = yVFolds
        
        xTest = xTrain[count]
        yTest = np.array(yTrain[count])
        
        np.delete(xTrain,count)
        np.delete(yTrain,count)
        
        yTrain = np.stack(yTrain)
        yTrain = yTrain.ravel()
        xTrainAux = []

        for i in range(kParts):
            for j in range(len(xTrain[i])):
                xTrainAux.append(xTrain[i][j])
        xTrain = np.stack(xTrainAux)
        yTest = np.where(yTest < 0, 0, yTest)
        predicted = algorithm(xTrain, yTrain, xTest,  *args)
        for i in range(len(predicted)):
            print("Predicted: ",predicted[i],"Actual: ",yTest[i])
        accuracy = np.array([x - y for x, y in zip(predicted, yTest)])/kParts
        scores.append(accuracy) 
    return scores

In [8]:
def sigmoide(z):
    return 1 / (1 + np.exp(-z))

In [9]:
def predict(xMatrix, w):
    predictions = []
    xMatrix = np.insert(xMatrix, 0, 1, axis =1)
    for i in range(xMatrix.shape[0]):
        predictions.append(round(sigmoide(xMatrix[i].dot(w))))
    return predictions

def logisticRegression(xTrain, yTrain, xTest, alpha, epochs): 
    coef = stochasticGradientDescent(xTrain, yTrain, epochs, alpha)
    return(predict(xTest, coef))

In [10]:
epochs = 1000 
alpha =  0.001
kParts = 5;

In [14]:
xMatrix, yVector = normalization(xMatrix, yVector)
kFoldCrossValidation(xMatrix, yVector, kParts, logisticRegression, alpha, epochs)

Predicted:  1.0 Actual:  1.0
Predicted:  1.0 Actual:  1.0
Predicted:  1.0 Actual:  1.0
Predicted:  1.0 Actual:  1.0
Predicted:  0.0 Actual:  0.0
Predicted:  0.0 Actual:  0.0
Predicted:  0.0 Actual:  0.0
Predicted:  1.0 Actual:  1.0
Predicted:  0.0 Actual:  0.0
Predicted:  0.0 Actual:  0.0
Predicted:  1.0 Actual:  1.0
Predicted:  0.0 Actual:  0.0
Predicted:  1.0 Actual:  1.0
Predicted:  0.0 Actual:  0.0
Predicted:  0.0 Actual:  0.0
Predicted:  1.0 Actual:  1.0
Predicted:  1.0 Actual:  1.0
Predicted:  1.0 Actual:  1.0
Predicted:  0.0 Actual:  0.0
Predicted:  1.0 Actual:  1.0
Predicted:  1.0 Actual:  1.0
Predicted:  1.0 Actual:  1.0
Predicted:  1.0 Actual:  1.0
Predicted:  1.0 Actual:  1.0
Predicted:  0.0 Actual:  0.0
Predicted:  0.0 Actual:  0.0
Predicted:  0.0 Actual:  0.0
Predicted:  1.0 Actual:  1.0
Predicted:  0.0 Actual:  0.0
Predicted:  0.0 Actual:  0.0
Predicted:  1.0 Actual:  1.0
Predicted:  0.0 Actual:  0.0
Predicted:  1.0 Actual:  1.0
Predicted:  0.0 Actual:  0.0
Predicted:  0.

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.])]

# Multiclass

In [15]:
dataTableMulticlass = pd.read_csv('iris_multiclasse.csv', names =['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'flower1','flower2','flower3'])

In [16]:
dataTableMulticlass.head()

,sepal_length,sepal_width,petal_length,petal_width,flower1,flower2,flower3
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


In [17]:
matrix_x_multiclass = np.array(dataTableMulticlass[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])
vector_y_multiclass = np.array(dataTableMulticlass[['flower1', 'flower2', 'flower3']])

In [18]:
matrix_x_multiclass, vector_y_multiclass = normalization(matrix_x_multiclass, vector_y_multiclass)

In [19]:
def gradientDescentMulticlass(matrix_x, vector_y, epochs, alpha):
    matrix_x = np.insert(matrix_x, 0, 1, axis =1)
    
    size = (vector_y.shape[1], matrix_x.shape[1])
    w_matrix = np.ones(size, dtype=float)

    mse = []
    for k in range(w_matrix.shape[0]): 
        y_aux = w_matrix[k].dot(np.transpose(matrix_x))

        for epoch in range(epochs):
            somatorio = 0
            for i in range(matrix_x.shape[0]):
                _sum = 0
                for j in range(w_matrix.shape[0]):
                    _sum += np.exp(w_matrix[j].dot(matrix_x[i]))
                
                err = np.exp(y_aux[i])/_sum
                somatorio += (vector_y[i][k] - err) * matrix_x[i]
            
            mse.append((-1/matrix_x.shape[0]) * somatorio)
            w_matrix[k] = w_matrix[k] + (alpha/matrix_x.shape[0])*somatorio
            y_aux = w_matrix[k].dot(np.transpose(matrix_x))


    return w_matrix, mse

In [21]:
def stochastic_gradient_descent_multiclass(matrix_x, vector_y, epochs, alpha):
    
    matrix_x = np.insert(matrix_x, 0, 1, axis =1)
    size = (vector_y.shape[1], matrix_x.shape[1])    
    w_matrix = np.ones(size, dtype=float)

    data =np.append(matrix_x, vector_y, axis=1)
    
    mse = []

    for k in range(w_matrix.shape[0]):
        y_aux = w_matrix[k].dot(np.transpose(matrix_x))

        for epoch in range(epochs):
            for i in range(matrix_x.shape[0]):
                _sum = 0
                for j in range(w_matrix.shape[0]):
                    _sum += np.exp(w_matrix[j].dot(matrix_x[i]))
        
                err = np.exp(y_aux[i])/_sum
                
                w_matrix[k] = w_matrix[k] + alpha * (vector_y[i][k] - err) * matrix_x[i]
      
            data = np.random.permutation(data)
            matrix_x = data[: ,0:matrix_x.shape[1]]
            vector_y = data[:,matrix_x.shape[1]:]
            y_aux = w_matrix[k].dot(np.transpose(matrix_x))
            
                
    return w_matrix, mse

In [22]:
def crossValidationSplitMulticlass(xMatrix, yVector, n_folds): 
    xMatrixSplit = []
    yVectorSplit = []
    xMatrixCopy = xMatrix

    yVectorCopy = yVector

    foldSize = int(len(xMatrix) / n_folds) 
    
    for i in range(n_folds): 
        foldXM = [] 
        foldYV = []
        while len(foldXM) < foldSize: 
            index = rd.randrange(len(xMatrixCopy)) 
            foldXM.append(xMatrixCopy[index]) 
            xMatrixCopy = np.delete(xMatrixCopy, index, axis=0)
            foldYV.append(yVectorCopy[index]) 
            yVectorCopy = np.delete(yVectorCopy, index,axis=0)
        xMatrixSplit.append(foldXM)
        yVectorSplit.append(foldYV)

    return xMatrixSplit, yVectorSplit

def kFoldCrossValidationMulticlass(xMatrix, yVector, kParts, algorithm, *args):
    xMFolds, yVFolds = crossValidationSplitMulticlass(xMatrix, yVector, kParts) 
    scores = list() 
    count = 0;
    for fold in xMFolds: 
        xTrain = xMFolds
        yTrain = yVFolds
        
        xTest = xTrain[count]
        yTest = np.array(yTrain[count])
        
        np.delete(xTrain,count)
        np.delete(yTrain,count)
        
        xTrainAux = []
        yTrainAux = []
        for i in range(kParts):
            for j in range(len(yTrain[i])):
                yTrainAux.append(yTrain[i][j])
        for i in range(kParts):
            for j in range(len(xTrain[i])):
                xTrainAux.append(xTrain[i][j])
        xTrain = np.stack(xTrainAux)
        yTrain = np.stack(yTrainAux)
        
        predicted = algorithm(xTrain, yTrain, xTest,  *args)
        for i in range(len(predicted)):
            aux = np.array([round(x) for x in yTest[i]])
            aux = np.where(aux < 0, 0, aux)
            print("Predicted: ",predicted[i],"Actual: ", aux)
        accuracy = np.array([x - y for x, y in zip(predicted, aux)])/kParts
        scores.append(accuracy) 
    return scores

In [23]:
def predict_multiclass_logistic_regression(matrix_x_multiclass, w):
    
    matrix_x_multiclass = np.insert(matrix_x_multiclass, 0, 1, axis =1)
    size = (matrix_x_multiclass.shape[0], w.shape[0])
    predictions = np.ones(size, dtype=float)
    
    for k in range(w.shape[0]):   
        for i in range(matrix_x_multiclass.shape[0]):
            num = np.exp(w[k].dot(matrix_x_multiclass[i]))
            sum_ = 0
            for j in range(w.shape[0]):
                sum_ = sum_ + np.exp(w[j].dot(matrix_x_multiclass[i]))
            predictions[i][k] = num/sum_

    for i in range(predictions.shape[0]):
        index = np.argmax(predictions[i], axis=0)
        for j in range(len(predictions[i])):
            if(j == index):
                predictions[i][index] = 1.0
            else:
                predictions[i][j] = 0.0
    
    return predictions

In [24]:
def logisticRegressionMulticlass(xTrain, yTrain, xTest, alpha, epochs): 
    #xTrain, yTrain = normalization(xTrain, yTrain)
    coef, mse = stochastic_gradient_descent_multiclass(xTrain, yTrain, epochs, alpha)
    return(predict_multiclass_logistic_regression(xTest, coef))

In [25]:
kFoldCrossValidationMulticlass(matrix_x_multiclass, vector_y_multiclass, kParts, logisticRegressionMulticlass, alpha, epochs)

Predicted:  [1. 0. 0.] Actual:  [1. 0. 0.]
Predicted:  [0. 0. 1.] Actual:  [0. 0. 1.]
Predicted:  [1. 0. 0.] Actual:  [1. 0. 0.]
Predicted:  [1. 0. 0.] Actual:  [1. 0. 0.]
Predicted:  [1. 0. 0.] Actual:  [1. 0. 0.]
Predicted:  [0. 0. 1.] Actual:  [0. 1. 0.]
Predicted:  [0. 1. 0.] Actual:  [0. 1. 0.]
Predicted:  [0. 0. 1.] Actual:  [0. 1. 0.]
Predicted:  [1. 0. 0.] Actual:  [1. 0. 0.]
Predicted:  [0. 0. 1.] Actual:  [0. 1. 0.]
Predicted:  [1. 0. 0.] Actual:  [1. 0. 0.]
Predicted:  [1. 0. 0.] Actual:  [1. 0. 0.]
Predicted:  [1. 0. 0.] Actual:  [1. 0. 0.]
Predicted:  [0. 1. 0.] Actual:  [0. 1. 0.]
Predicted:  [0. 0. 1.] Actual:  [0. 0. 1.]
Predicted:  [0. 1. 0.] Actual:  [0. 1. 0.]
Predicted:  [0. 0. 1.] Actual:  [0. 1. 0.]
Predicted:  [0. 0. 1.] Actual:  [0. 0. 1.]
Predicted:  [0. 1. 0.] Actual:  [0. 1. 0.]
Predicted:  [1. 0. 0.] Actual:  [1. 0. 0.]
Predicted:  [0. 0. 1.] Actual:  [0. 1. 0.]
Predicted:  [1. 0. 0.] Actual:  [1. 0. 0.]
Predicted:  [0. 0. 1.] Actual:  [0. 0. 1.]
Predicted: 

[array([[ 0.2,  0. ,  0. ],
        [ 0. ,  0. ,  0.2],
        [ 0. , -0.2, -0.2]]), array([[ 0.2,  0. ,  0. ],
        [ 0. ,  0. ,  0.2],
        [ 0. , -0.2, -0.2]]), array([[ 0.2,  0. ,  0. ],
        [ 0. ,  0. ,  0.2],
        [ 0. , -0.2, -0.2]]), array([[ 0.2,  0. ,  0. ],
        [ 0. ,  0. ,  0.2],
        [ 0. , -0.2, -0.2]]), array([[ 0.2,  0. ,  0. ],
        [ 0. ,  0. ,  0.2],
        [ 0. , -0.2, -0.2]])]